In [21]:
from s3i import UserMessage,IdentityProvider, TokenType, Broker, Key, Directory
import os
import uuid
import base64

%store -r

# Get Token


In [23]:
%store -r
#idp = IdentityProvider(grant_type='password', identity_provider_url="https://idp.s3i.vswf.dev/", realm='KWH',
#                                             client_id=CLIENT, client_secret=SECRET, username=USERNAME, password=PASSWORD)
print("[S3I]: Access token: \n ", IDP.get_token(TokenType.ACCESS_TOKEN))

[S3I]: Access token: 
  eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJPbzRfUTE0UGxjeVJfX1A5MXAtYXRaWURYa29GVV9ORk5qLWtLdDVHdXNJIn0.eyJqdGkiOiIxNTkwYzJjOS0wNWRkLTQ3NGYtODViYy1kZGJlNTJmMGI0MmUiLCJleHAiOjE1ODk4Mjc3MTUsIm5iZiI6MCwiaWF0IjoxNTg5ODI3NDE1LCJpc3MiOiJodHRwczovL2lkcC5zM2kudnN3Zi5kZXYvYXV0aC9yZWFsbXMvS1dIIiwiYXVkIjoicmFiYml0bXEiLCJzdWIiOiI2MDZkOGIzOC00YzNmLTQ2YmQtOTQ4Mi04Njc0OGUxMDhmMzIiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJzM2k6NGYwNzYxMjctZDllMi00NzdkLThkNDItNDQ1MzIyNWJmYWFhIiwiYXV0aF90aW1lIjowLCJzZXNzaW9uX3N0YXRlIjoiMmQ3ZDIyMWUtZDlmNi00NjVlLTllOWItMmU2MjFkOWJkODA3IiwiYWNyIjoiMSIsInNjb3BlIjoib3BlbmlkIHJhYmJpdG1xLndyaXRlOiovKi8qIHByb2ZpbGUgcmFiYml0bXEucmVhZDoqLyovKiByYWJiaXRtcS50YWc6bW9uaXRvcmluZyBlbWFpbCIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwicHJlZmVycmVkX3VzZXJuYW1lIjoia3doLXRlYW0ifQ.cGNjhsQvQy2zCDCPdUXID1930hMvnFQfm0as8Dn1rTyVSeDP5CPfpVP7sV3Gl36R9NO32GUz86-ZPdxsxvX9YbRPZhz9VzENs_KR4oPMyi2MoBsAisvUtpaRYieBji-hasHe3Od4lslOJTDCJIqvgKFxboegDzQxZ0wTkJCVDTRXzbFiRr-CWMJdbK894KbQHUozq-iXT16on

# Send User Messages

## Verify the endpoints

In [29]:
receivers = [str(x) for x in input('[S3I]: Please enter the receivers, seperate with a space:').split()]


[S3I]: Please enter the receivers, seperate with a space:s3i:bf5a3b54-1ecc-4195-bc55-cb4757b895d0


In [30]:
#idp = IdentityProvider(grant_type='password', identity_provider_url="https://idp.s3i.vswf.dev/", realm='KWH',
#                                             client_id=CLIENT, client_secret=SECRET, username=USERNAME, password=PASSWORD)
access_token = IDP.get_token(TokenType.ACCESS_TOKEN)
dir = Directory(s3i_dir_url="https://dir.s3i.vswf.dev/api/2/", token=access_token)
defaultHMIs = [dir.queryThingIDBased(receiver+"/attributes/defaultHMI") for receiver in receivers]
endpoints = [dir.queryThingIDBased(hmi+"/attributes/allEndpoints")[0] for hmi in defaultHMIs]

In [ ]:
endpoints = [str(x) for x in input('[S3I]: Please enter the receivers, seperate with a space:').split()]

In [ ]:
#receivers_endpoints = #["s3ibs://s3i:bf5a3b54-1ecc-4195-bc55-cb4757b895d0"]
#IDP = IdentityProvider(grant_type='password', identity_provider_url="https://idp.s3i.vswf.dev/", realm='KWH',
#                                             client_id=CLIENT, client_secret=SECRET, username=USERNAME, password=PASSWORD)
#access_token = IDP.get_token(TokenType.ACCESS_TOKEN)
#broker = Broker(auth_form='Username/Password', username=" ", password=access_token, host="rabbitmq.s3i.vswf.dev")
#msg = UserMessage(sender=THINGID, identifier="apifruzt",receivers=receivers_endpoints, subject="Test", text="TestText")

#broker.send(receivers_endpoints,msg.msg.__str__())

## Prepare the message

In [62]:
subject = input('[S3I]: please enter the subject: \n')
text = input('[S3I]: please enter the text: \n')
msg_uuid = "s3i:" + str(uuid.uuid4())
replyToEndpoint=None
attachments=None

[S3I]: please enter the subject: 
Jannik2
[S3I]: please enter the text: 
Haeschen


### Add attachement to message

In [25]:
filename_list = list(map(str, input("[S3I]: Please enter the name of file that you want to send: ").split()))
attachments = list()

for filename in filename_list:
    attachment_dict = dict()
    attachment_dict["filename"] = filename
    with open("attachment_files/"+filename, "rb") as f:
        # encode an attachment file to BASE64 bytes
        base64_data = base64.b64encode(f.read())
        data = base64_data.decode()  # convert byte to str
        attachment_dict["data"] = data
        attachments.append(attachment_dict)

[S3I]: Please enter the name of file that you want to send: text.txt


## Assemble User Message

In [26]:
message = UserMessage(sender=CLIENT, identifier=msg_uuid,receivers=receivers, subject=subject, text=text,
                  replyToEndpoint=replyToEndpoint, attachments=attachments)
print(message.msg)

{'sender': 's3i:9b34f12b-f0ab-4f13-b0cc-82202e7a9770', 'identifier': 's3i:8f9eb115-c2fa-4aa8-b411-ea962e931c29', 'receivers': ['s3i:bf5a3b54-1ecc-4195-bc55-cb4757b895d0'], 'messageType': 'userMessage', 'replyToEndpoint': '', 'attachments': [{'filename': 'text.txt', 'data': 'VGhpcyBpcyBhbiBhdHRhY2htZW50'}], 'subject': 'Kannik', 'text': 'Hase2'}


In [65]:
pgpKey_sec = Key(path_demo_dir="key", filename="test_sec.asc")
message.sign(pgpKey_sec.key)
print(message.pgpMsg.__str__())

-----BEGIN PGP MESSAGE-----

yMG+ATvCy8zAxbiet0p4v+a0fIbTB7aXMsQduvC1Wr04NS8ltUjdSkG92DjTyiTN
wNzM0MhcN8Uy1UjXxNw8RdcixQTIMjE1NjIyTUpLTExU11FQz0xJzSvJTMtE6DRM
S7awNE4z1jVPMUrUNUk0S9NNMjZJ0U00STUwSEk1TTO2NAbpLEpNTs0sSy0qBmqM
ButMSjNNNE4yNdE1TE1O1jUxtDTVTUo2NdVNTjIxNzVPsrA0TTFQjwVqzU0tLk5M
Tw2pLEgF2VpanFrkCxGCGFyQUxmS75qXUpCfmVcCUgESTiwpSUzOyAU6F2wjyJzi
0qSs1GSwCq/EvLzMbCOQwpLUCrCQR2JqcXJGah5Ccypcd7V6WmZOal5iLtgBIB16
JUBdQJUpiSWJILEw94yC5EonEM5IynTKSPEIyog0yiiJCjc1UK+NrT10oJCFgZGL
QZqVCRj+v0VEGhiAgI2BFeISj/ySkkoGLk4BWFQ9fcnBsKjGcltn0KZFVfOCOK5P
2z5Bt7m3r0WuY0ZCXbblneBuyeA9PzalNL+9HNAZuSLpnd6O7/Ymm54KZ3t6Tg9y
LJH5VVtf9Cr4c0V2htNy1YZFp7ibJ07m3W7tMuFThH3TSzH3BfH8e7U3yuQo5Djc
vh4q0XWs7PDqI6dXr/Q5+TjNaivbvg/vVeoF+97cCp9ovcP5gXrpE5UlrQk/X286
zC4rKv3WNn7B/tQ7U2fY8sy8u5ajdD7DohSfTPZ1vEbzfF5+N6xcHOjxUGLrU9eF
9W8um/rt335UrOr0X40G04+B0cerUpUqN3+Vsc3+H1V41i5m16mkT2kbdov+NZvH
+627V+7XlIzXajtuMBv5AQA=
=pilJ
-----END PGP MESSAGE-----



In [27]:
IDP = IdentityProvider(grant_type='password', identity_provider_url="https://idp.s3i.vswf.dev/", realm='KWH',
                                             client_id=CLIENT, client_secret=SECRET, username=USERNAME, password=PASSWORD)
access_token = IDP.get_token(TokenType.ACCESS_TOKEN)
dir = Directory(s3i_dir_url="https://dir.s3i.vswf.dev/api/2/", token=access_token)
publicKeys_str = dir.getPublicKey(receivers)
#pubKey = Key(key_str = publicKeys)
#message.encrypt(publicKeys)
#print(message.pgpMsg.__str__())
#key_str_list = s3i_directory.getPublicKey(receivers)
publicKeys = [Key(key_str=i) for i in publicKeys_str]
message.encrypt(publicKeys)
#print(message.pgpMsg.__str__())

In [28]:
IDP = IdentityProvider(grant_type='password', identity_provider_url="https://idp.s3i.vswf.dev/", realm='KWH',
                                             client_id=CLIENT, client_secret=SECRET, username=USERNAME, password=PASSWORD)
access_token = IDP.get_token(TokenType.ACCESS_TOKEN)
broker = Broker(auth_form='Username/Password', username=" ", password=access_token, host="rabbitmq.s3i.vswf.dev")
broker.send(endpoints, message.pgpMsg.__str__())